# Credit Score Project

---

## Importing the data

In [548]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle, islice
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [549]:
# Display dataset with no index column
raw_df = pd.read_csv('train_biased.csv')
df = raw_df.copy()
raw_df

,ID,Customer_ID,Month,Name,City,Street,Age,SSN,Occupation,Annual_Income,...,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Credit_Score
0,NaN,CUS_0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,NaN,821-00-0265,Manager,19114.12,...,7,11.27,4.0,_,NaN,26.822620,22 Years and 1 Months,No,80.41529543900253,3
1,0x1603,CUS_0xd40,February,Aaron Maashoh,Lonton,Oxford Street,23.0,821-00-0265,Manager,19114.12,...,NaN,11.27,4.0,NaN,809.98,31.944960,NaN,No,118.28022162236736,3
2,0x1604,CUS_0xd40,NaN,NaN,Lonton,Oxford Street,-500.0,821-00-0265,Manager,19114.12,...,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648,3
3,0x1605,CUS_0xd40,April,Aaron Maashoh,Lonton,Oxford Street,23.0,821-00-0265,Manager,19114.12,...,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713,3
4,NaN,CUS_0xd40,May,Aaron Maashoh,Lonton,Oxford Street,23.0,821-00-0265,Manager,19114.12,...,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,BadShire,North Street,NaN,078-73-5990,Cleaner,39628.99,...,7,11.5,NaN,_,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485,1
99996,0x25fea,CUS_0x942c,May,Nicks,BadShire,North Street,25.0,078-73-5990,Cleaner,39628.99,...,7,11.5,3.0,_,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385,1
99997,0x25feb,NaN,June,Nicks,BadShire,North Street,25.0,078-73-5990,Cleaner,NaN,...,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441,1
99998,0x25fec,CUS_0x942c,July,Nicks,Standhampton,North Street,25.0,078-73-5990,Cleaner,39628.99,...,NaN,11.5,3.0,Good,NaN,33.638208,31 Years and 9 Months,No,251.67258219721603,2


--- 
## EDA (Pre Data Handling)

---

## Handling Missing and Incorrect Values

In [550]:
# Create a copy of the 'Credit_Score' column
credit_score_df = raw_df['Credit_Score'].copy()

# Remove the 'ID', 'SSN' and 'Name' column from the DataFrame
df.drop(columns=['ID', 'SSN', 'Credit_Score'], inplace=True, axis=1)
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS_0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,_,NaN,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS_0xd40,February,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,NaN,809.98,31.944960,NaN,No,118.28022162236736
2,CUS_0xd40,NaN,NaN,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648
3,CUS_0xd40,April,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS_0xd40,May,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS_0x942c,April,Nicks,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,_,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS_0x942c,May,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,_,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,NaN,June,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS_0x942c,July,Nicks,Standhampton,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,NaN,33.638208,31 Years and 9 Months,No,251.67258219721603


In [551]:
# Find all the columns that contain underscores in their values
underscore_columns = []
for col in df.columns:
    # Check if column contains any string values
    if df[col].dtype == 'object' and df[col].astype(str).str.contains('_').any():
            underscore_columns.append(col)

print("Columns with underscores:", underscore_columns)

# Clean underscores from all columns identified except 'Customer_ID' and 'Occupation' (assuming these are the columns you want to keep underscores in)
for col in underscore_columns:
    if col != 'Customer_ID' or col != 'Occupation':
        df[col] = df[col].apply(lambda x: x.replace('_','') if isinstance(x, str) else x)

# Fix "Credit_Mix" empty strings
df['Credit_Mix'] = df['Credit_Mix'].replace('', np.nan)
df

Columns with underscores: ['Customer_ID', 'Occupation', 'Annual_Income', 'Num_of_Loan', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Credit_Mix', 'Outstanding_Debt', 'Amount_invested_monthly']


,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,NaN,NaN,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,February,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,NaN,809.98,31.944960,NaN,No,118.28022162236736
2,CUS0xd40,NaN,NaN,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648
3,CUS0xd40,April,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,May,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,April,Nicks,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,NaN,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,May,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,NaN,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,NaN,June,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,July,Nicks,Standhampton,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,NaN,33.638208,31 Years and 9 Months,No,251.67258219721603


In [552]:
# Finding the uniform columns in the DataFrame
def uniform_columns(df, chunk_size=8):
    chunks_total = len(df) // chunk_size
    uniform_columns = []

    for col in df.columns:
        uniform_count = 0
        for i in range(chunks_total):
            chunk = df.iloc[i*chunk_size:(i+1)*chunk_size]
            non_na_values = set(chunk[col].dropna())
            if len(non_na_values) <= 1:
                uniform_count += 1
        if uniform_count == chunks_total:
            uniform_columns.append(col)
    return uniform_columns

uniform_col = uniform_columns(df,8)
uniform_col

['Customer_ID',
 'Name',
 'Street',
 'Occupation',
 'Type_of_Loan',
 'Credit_Mix',
 'Outstanding_Debt']

### Customer ID

In [553]:
# Process the DataFrame in chunks of 8 rows
chunk_size = 8

# Fill the Customer_ID column with the mode value of each chunk
for start in range(0, len(df), chunk_size):
    end = start + chunk_size
    chunk_indices = df.index[start:end]
    chunk = df.loc[chunk_indices]
    # For each specified column, calculate the mode and update the chunk rows
    mode_value = chunk['Customer_ID'].mode().iloc[0]
    df.loc[chunk_indices, 'Customer_ID'] = mode_value
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,NaN,NaN,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,February,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,NaN,809.98,31.944960,NaN,No,118.28022162236736
2,CUS0xd40,NaN,NaN,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648
3,CUS0xd40,April,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,May,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,April,Nicks,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,NaN,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,May,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,NaN,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,June,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,July,Nicks,Standhampton,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,NaN,33.638208,31 Years and 9 Months,No,251.67258219721603


### Uniform Columns (Name, Street, Occupation, Type_of_Loan, Outstanding_Debt)

In [554]:
# Create a copy of the DataFrame with only the uniform columns
uniform_df = df[uniform_col].copy()

# For each uniform column fill the missing values with the mode of each grouped Customer_ID
for col in uniform_df.columns:
    if col != 'Customer_ID':
        uniform_df[col] = uniform_df.groupby('Customer_ID')[col].transform(
            lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)
        df[col] = uniform_df[col]
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,February,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,118.28022162236736
2,CUS0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648
3,CUS0xd40,April,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,May,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,April,Nicks,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,May,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,June,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,July,Nicks,Standhampton,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Num of Loans

In [555]:
# Remake the Num_of_Loan column by counting each loan type in the Type_of_Loan column
df['Num_of_Loan'] = df['Type_of_Loan'].fillna("").apply(
    lambda x: len([item for item in x.split(",") if item.strip()]))
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,February,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,118.28022162236736
2,CUS0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648
3,CUS0xd40,April,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,May,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,April,Nicks,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,May,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,June,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,July,Nicks,Standhampton,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Type of Loans

In [556]:
# Fill fixed NaN values with "No Loan"
df['Type_of_Loan'] = df['Type_of_Loan'].replace(np.nan, "No Loan")
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,February,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,118.28022162236736
2,CUS0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648
3,CUS0xd40,April,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,May,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,April,Nicks,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,May,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,June,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,July,Nicks,Standhampton,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### City

In [557]:
# Considered that 'Street' is a uniform column we can fill the missing values in 'City' in the same way and fix the errors in the 'City' column
city_df = df[['Customer_ID', 'City']].copy()

# First, get the mode of 'City' for each customer ID group
city_modes = city_df.groupby('Customer_ID')['City'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

# Marking error values that don't match the mode of the group as NaN
city_df['City'] = city_df.apply(lambda x: 
    np.nan if (pd.notnull(x['City']) and x['City'] != city_modes[x['Customer_ID']]) 
    else x['City'], axis=1)

# Filling all NaN values with the mode of the group
city_df['City'] = city_df.apply(lambda x: 
    city_modes[x['Customer_ID']] if (pd.isnull(x['City']) and x['Customer_ID'] in city_modes)
    else x['City'], axis=1)

# Updating the original DataFrame with the cleaned 'City' column
df['City'] = city_df['City']
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,February,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,118.28022162236736
2,CUS0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648
3,CUS0xd40,April,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,May,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,April,Nicks,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,May,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,June,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,July,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Month

In [558]:
# Define month encoding
month_to_number = { "January": 1,
                    "February": 2,
                    "March": 3,
                    "April": 4,
                    "May": 5,
                    "June": 6,
                    "July": 7,
                    "August": 8}

# Encode the known months
df["Month"] = df["Month"].map(month_to_number)

# Fill in the missing values using the repeating sequence 1-8
encoded_cycle = list(islice(cycle(range(1, 9)), len(df)))
df["Month"] = encoded_cycle
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Credit_History_Age

In [559]:
# Turning strings into (years, months)
def parse_age(age_str):
    if pd.isna(age_str):
        return np.nan, np.nan
    parts = age_str.split() # should be for example ["22", "Years", "and", "1", "Months"]
    return int(parts[0]), int(parts[3])

def format_age(years, months):
    return f"{years} Years and {months} Months"

# Determine number of groups (each group has 8 rows)
n_groups = len(df) // chunk_size

# Process each group
for i in range(n_groups):
    start = i * chunk_size
    end = start + chunk_size
    group = df.iloc[start:end]
    
    # Extract years and months for the non-missing values
    years_list = []
    months_list = []
    indices = []
    for idx, row in group.iterrows():
        years, months = parse_age(row['Credit_History_Age'])
        if not (pd.isna(years) or pd.isna(months)):
            years_list.append(years)
            months_list.append(months)
            indices.append(idx)
    
    # If we have at least two known points, we can linearly interpolate by index
    if len(indices) >= 2:
        # Create arrays for interpolation over the index positions (0 to 7)
        pos = np.array(indices)
        # We assume the interpolation is linear with respect to the row number
        # Interpolate years and months separately
        all_pos = np.arange(start, end)
        # Interpolate years
        years_interp = np.interp(all_pos, pos, years_list)
        # Interpolate months
        months_interp = np.interp(all_pos, pos, months_list)
        
        # Round the results (since ages are integers)
        years_interp = np.round(years_interp).astype(int)
        months_interp = np.round(months_interp).astype(int)
        
        # Update the dataframe for rows with NaN
        for pos_idx, row_idx in enumerate(range(start, end)):
            if pd.isna(df.loc[row_idx, 'Credit_History_Age']):
                df.loc[row_idx, 'Credit_History_Age'] = format_age(years_interp[pos_idx], months_interp[pos_idx])
    else:
        # If not enough points to interpolate, we might use forward fill if available
        df.iloc[start:end, df.columns.get_loc('Credit_History_Age')] = group['Credit_History_Age'].ffill().bfill()

df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Payment_of_Min_Amount

In [560]:
# Fix the errors in the 'Payment_of_Min_Amount' column
payment_df = df[['Customer_ID', 'Payment_of_Min_Amount']].copy()

# First, get the mode of 'Payment_of_Min_Amount' for each customer ID group
payment_modes = payment_df.groupby('Customer_ID')['Payment_of_Min_Amount'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

# Marking error values that don't match the mode of the group as NaN
payment_df['Payment_of_Min_Amount'] = payment_df.apply(lambda row: 
    np.nan if (pd.notnull(row['Payment_of_Min_Amount']) and 
                row['Payment_of_Min_Amount'] != payment_modes[row['Customer_ID']]) 
    else row['Payment_of_Min_Amount'], axis=1)

# Filling all NaN values with the mode of the group
payment_df['Payment_of_Min_Amount'] = payment_df.apply(lambda row: 
    payment_modes[row['Customer_ID']] if (pd.isnull(row['Payment_of_Min_Amount']) and 
                                        row['Customer_ID'] in payment_modes)
    else row['Payment_of_Min_Amount'], axis=1)

# Updating the original DataFrame with the cleaned 'Payment_of_Min_Amount' column
df['Payment_of_Min_Amount'] = payment_df['Payment_of_Min_Amount']
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Age

In [561]:
# Fix the errors in the 'Age' column
age_df = df[['Customer_ID', 'Age']].copy()

def age_fix(age_series):
    corrected = []
    age1 = None  # The first valid age
    age2 = None  # The second valid age 

    for val in age_series:
        # Decide whether this entry is valid
        if pd.isna(val) or val < 13 or val > 120:
            # invalid or missing -> fill with known age if possible
            if age1 is None:
                # no known valid age yet
                corrected.append(np.nan)
            elif age2 is None:
                # we've only established age1 so far
                corrected.append(age1)
            else:
                # we've already switched to age2
                corrected.append(age2)
        else:
            # val is a valid age
            if age1 is None:
                # first valid age we see
                age1 = val
                corrected.append(val)
            else:
                # already have age1
                if val == age1:
                    # same as first age
                    if age2 is None:
                        corrected.append(age1)
                    else:
                        # we already switched to age2, so ignore going "back" to age1
                        corrected.append(age2)
                else:
                    # val is a different age
                    if age2 is None:
                        # first time we see a new age -> set age2
                        age2 = val
                        corrected.append(val)
                    else:
                        # can't have a third age -> stick to age2
                        corrected.append(age2)
    # Fill in leading/trailing NaNs
    corrected = pd.Series(corrected).ffill().bfill().values
    return corrected

# Loop explicitly over each group by Customer_ID and update the age_df DataFrame
for customer, group in age_df.groupby('Customer_ID'):
    group_index = group.index
    corrected_values = age_fix(group['Age'])
    age_df.loc[group_index, 'Age'] = corrected_values

# Updating the original DataFrame with the cleaned 'Age' column
df['Age'] = age_df['Age']
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Annual Income

In [562]:
# Copy the 'Annual_Income' column to a new DataFrame
df_annual = df[['Customer_ID','Annual_Income']].copy()

# Transform the 'Annual_Income' column to float
df_annual['Annual_Income'] = df_annual['Annual_Income'].astype(float).round(2)

distribution_df = df_annual["Annual_Income"].value_counts().sort_index().reset_index()
distribution_df.columns = ['value', 'count']

# show the average value of the groupes that have the same count
average_distribution = distribution_df.groupby('count')['value'].mean().reset_index().round(2)
average_distribution.columns = ['count', 'average_value']

# Column with number of times each count appears
average_distribution['count_appearances'] = average_distribution['count'].map(
    distribution_df['count'].value_counts().to_dict()
)
average_distribution

,count,average_value,count_appearances
0,1,12576843.76,905
1,3,59084.38,5
2,4,54363.22,73
3,5,51837.77,570
4,6,50079.30,2077
5,7,51169.93,4795
6,8,49891.45,4952
7,12,17816.75,1
8,13,36585.12,1
9,14,32766.68,6


In [563]:
# Identify Annual_Income values with frequency below 3
value_counts = df_annual["Annual_Income"].value_counts()
to_nan = value_counts[value_counts < 3].index

# Replace those values with pd.NA in df_annual
df_annual.loc[df_annual["Annual_Income"].isin(to_nan), "Annual_Income"] = pd.NA

# Check if the column is uniform
uniform_columns(df_annual, 8)

['Customer_ID', 'Annual_Income']

In [564]:
# For each uniform column fill the missing values with the mode of each grouped Customer_ID
df_annual["Annual_Income"] = df_annual.groupby("Customer_ID")["Annual_Income"].transform(
            lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

# Fill the missing values in the original DataFrame with the cleaned values
df["Annual_Income"] = df_annual["Annual_Income"]
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.415833,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,NaN,NaN,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Monthly Inhand Salary

In [565]:
# Create a copy of the 'Monthly_Inhand_Salary' column
df_month = df[["Monthly_Inhand_Salary"]].copy().astype(float).round(2)

# Check how many columns are uniform in the column 'Monthly_Inhand_Salary'
uniform_count = 0
chunks_total = len(df_month) // chunk_size
for i in range(chunks_total):
    chunk = df_month.iloc[i*chunk_size:(i+1)*chunk_size]
    non_na_values = set(chunk["Monthly_Inhand_Salary"].dropna())
    # Count as uniform if there are no non-NaN values or exactly one unique value.
    if len(non_na_values) <= 1:
        uniform_count += 1
non_uniform_count = chunks_total - uniform_count
print(f"Uniform Chunks = {uniform_count}, Non-Uniform Chunks = {non_uniform_count}")

Uniform Chunks = 11785, Non-Uniform Chunks = 715


In [566]:
value_count_distribution = {}

for start in range(0, len(df_month), chunk_size):
    chunk = df_month.iloc[start:start+chunk_size]
    # Count the distinct non-NaN values in the chunk for the "Annual_Income" column
    possible_values = set(chunk["Monthly_Inhand_Salary"].dropna())
    count = len(possible_values)
    value_count_distribution[count] = value_count_distribution.get(count, 0) + 1

# Print the results sorted by the number of possible values.
for possible_val, chunk_count in sorted(value_count_distribution.items()):
    print(f"{chunk_count} chunks with {possible_val} possible value{'s' if possible_val != 1 else ''}")

11785 chunks with 1 possible value
715 chunks with 2 possible values


In [567]:
# Create a DataFrame with Customer_ID and Monthly_Inhand_Salary
df_month = df[['Customer_ID', 'Monthly_Inhand_Salary']].copy()
df_month['Monthly_Inhand_Salary'] = df_month['Monthly_Inhand_Salary'].astype(float).round(2)

def NaN_imputer(series):
    corrected = []
    first_valid = None  # The first valid salary
    second_valid = None  # The second valid salary

    for val in series:
        # Define an invalid salary (modify this condition as needed)
        if pd.isna(val) or val < 0:
            if first_valid is None:
                corrected.append(np.nan)
            elif second_valid is None:
                corrected.append(first_valid)
            else:
                corrected.append(second_valid)
        else:
            if first_valid is None:
                first_valid = val
                corrected.append(val)
            else:
                if val == first_valid:
                    if second_valid is None:
                        corrected.append(first_valid)
                    else:
                        corrected.append(second_valid)
                else:
                    if second_valid is None:
                        second_valid = val
                        corrected.append(val)
                    else:
                        corrected.append(second_valid)
    # Fill in leading/trailing NaNs if any
    corrected = pd.Series(corrected).ffill().bfill().values
    return corrected

# Process each group by 'Customer_ID'
for customer, group in df_month.groupby('Customer_ID'):
    group_index = group.index
    corrected_values = NaN_imputer(group['Monthly_Inhand_Salary'])
    df_month.loc[group_index, 'Monthly_Inhand_Salary'] = corrected_values

# Update the original DataFrame's column with the cleaned values   
df['Monthly_Inhand_Salary'] = df_month['Monthly_Inhand_Salary']
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,NaN,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Num Bank Accounts

In [568]:
df_bank_account = df[['Num_Bank_Accounts']]
df_bank_account.loc[(df_bank_account['Num_Bank_Accounts'] > 11) | (df_bank_account['Num_Bank_Accounts'] < 0),'Num_Bank_Accounts'] = np.nan
df["Num_Bank_Accounts"] = df_bank_account["Num_Bank_Accounts"]

/var/folders/k_/s1y99_w10p160dfzw5445lwh0000gn/T/ipykernel_80917/824910121.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bank_account.loc[(df_bank_account['Num_Bank_Accounts'] > 11) | (df_bank_account['Num_Bank_Accounts'] < 0),'Num_Bank_Accounts'] = np.nan


In [569]:
df_bank_account = df[['Customer_ID', 'Num_Bank_Accounts']].copy()

for customer, group in df_bank_account.groupby('Customer_ID'):
    group_index = group.index
    corrected_values = NaN_imputer(group['Num_Bank_Accounts'])
    df_bank_account.loc[group_index, 'Num_Bank_Accounts'] = corrected_values

df['Num_Bank_Accounts'] = df_bank_account['Num_Bank_Accounts']
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Num Credit card

In [570]:
df_Credit_Card = df[['Num_Credit_Card']]

df_Credit_Card.loc[
    (df_Credit_Card['Num_Credit_Card'] > 11) | (df_Credit_Card['Num_Credit_Card'] < 0),
    'Num_Credit_Card'
] = np.nan


/var/folders/k_/s1y99_w10p160dfzw5445lwh0000gn/T/ipykernel_80917/3067772878.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Credit_Card.loc[


In [571]:
value_count_distribution = {}

for start in range(0, len(df_Credit_Card), chunk_size):
    chunk = df_Credit_Card.iloc[start:start+chunk_size]
    # Count the distinct non-NaN values in the chunk for the "Annual_Income" column
    possible_values = set(chunk["Num_Credit_Card"].dropna())
    count = len(possible_values)
    value_count_distribution[count] = value_count_distribution.get(count, 0) + 1

# Print the results sorted by the number of possible values.
for possible_val, chunk_count in sorted(value_count_distribution.items()):
    print(f"{chunk_count} chunks with {possible_val} possible value{'s' if possible_val != 1 else ''}")

df["Num_Credit_Card"] = df_Credit_Card['Num_Credit_Card']

12199 chunks with 1 possible value
301 chunks with 2 possible values


In [572]:
# Create a DataFrame with Customer_ID and 
df_num_credit_card = df[['Customer_ID', 'Num_Credit_Card']].copy()

# Process each group by 'Customer_ID'
for customer, group in df_num_credit_card.groupby('Customer_ID'):
    group_index = group.index
    corrected_values = NaN_imputer(group['Num_Credit_Card'])
    df_num_credit_card.loc[group_index, 'Num_Credit_Card'] = corrected_values

# Update the original DataFrame's column with the cleaned values   
df['Num_Credit_Card'] = df_num_credit_card['Num_Credit_Card']
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Interest Rate

In [573]:
df_Interest = df[['Interest_Rate']]
df_Interest.loc[(df_Interest['Interest_Rate'] > 35),'Interest_Rate'] = np.nan

# Create a DataFrame with Customer_ID and 
df_Interest = df[['Customer_ID', 'Interest_Rate']].copy()

# Process each group by 'Customer_ID'
for customer, group in df_Interest.groupby('Customer_ID'):
    group_index = group.index
    corrected_values = NaN_imputer(group['Interest_Rate'])
    df_Interest.loc[group_index, 'Interest_Rate'] = corrected_values

# Update the original DataFrame's column with the cleaned values   
df['Interest_Rate'] = df_Interest['Interest_Rate']
df

/var/folders/k_/s1y99_w10p160dfzw5445lwh0000gn/T/ipykernel_80917/487928979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Interest.loc[(df_Interest['Interest_Rate'] > 35),'Interest_Rate'] = np.nan


,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Delay from due date

In [574]:
df_delay_date = df[['Delay_from_due_date']]

def impute_chunk_mean_round0(df, chunk_size=8):
    imputed_df = pd.DataFrame()
    for start in range(0, len(df), chunk_size):
        chunk = df.iloc[start:start+chunk_size].copy()
        # Select numeric columns in the chunk
        num_cols = chunk.select_dtypes(include=[np.number]).columns
        if len(num_cols) > 0:
            imputer = SimpleImputer(strategy='mean')
            # Impute missing values with the mean of the non-NaN values in the chunk
            chunk[num_cols] = imputer.fit_transform(chunk[num_cols])
            # Round imputed values to the nearest unit (0 decimals)
            chunk[num_cols] = chunk[num_cols].round(0)
        imputed_df = pd.concat([imputed_df, chunk])
    return imputed_df

df_delay_date = impute_chunk_mean_round0(df_delay_date, chunk_size=8)

df["Delay_from_due_date"] = df_delay_date["Delay_from_due_date"]
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,-1.0,NaN,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7,NaN,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,5.0,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,6.0,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,23.0,7,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,18.0,7,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,27.0,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,20.0,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Num of Delayed Payment

In [575]:
df_Delayed_Pay = df[['Num_of_Delayed_Payment']]
df_Delayed_Pay['Num_of_Delayed_Payment'] = pd.to_numeric(df_Delayed_Pay['Num_of_Delayed_Payment'], errors='coerce')
df_Delayed_Pay.loc[
    (df_Delayed_Pay['Num_of_Delayed_Payment'] < 0) | (df_Delayed_Pay['Num_of_Delayed_Payment'] > 29),
    'Num_of_Delayed_Payment'
] = np.nan
df_Delayed_Pay = impute_chunk_mean_round0(df_Delayed_Pay, chunk_size=8)
df["Num_of_Delayed_Payment"] = df_Delayed_Pay["Num_of_Delayed_Payment"]
df

/var/folders/k_/s1y99_w10p160dfzw5445lwh0000gn/T/ipykernel_80917/3641052089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Delayed_Pay['Num_of_Delayed_Payment'] = pd.to_numeric(df_Delayed_Pay['Num_of_Delayed_Payment'], errors='coerce')
/var/folders/k_/s1y99_w10p160dfzw5445lwh0000gn/T/ipykernel_80917/3641052089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Delayed_Pay.loc[


,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7.0,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,-1.0,6.0,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7.0,NaN,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,5.0,4.0,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,6.0,6.0,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,23.0,7.0,11.5,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,18.0,7.0,11.5,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,27.0,6.0,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,20.0,6.0,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Changed Credit Limit

In [576]:
df_changed_credit_limit = df[['Changed_Credit_Limit']]
df_changed_credit_limit['Changed_Credit_Limit'] = df_changed_credit_limit['Changed_Credit_Limit'].replace("", np.nan)
df_changed_credit_limit['Changed_Credit_Limit'] = df_changed_credit_limit['Changed_Credit_Limit'].astype(float)


def impute_chunk_mean_round2(df, chunk_size=8):
    imputed_df = pd.DataFrame()
    for start in range(0, len(df), chunk_size):
        chunk = df.iloc[start:start+chunk_size].copy()
        # Select numeric columns in the chunk
        num_cols = chunk.select_dtypes(include=[np.number]).columns
        if len(num_cols) > 0:
            imputer = SimpleImputer(strategy='mean')
            # Impute missing values with the mean of the non-NaN values in the chunk
            chunk[num_cols] = imputer.fit_transform(chunk[num_cols])
            # Round imputed values to the nearest unit (0 decimals)
            chunk[num_cols] = chunk[num_cols].round(2)
        imputed_df = pd.concat([imputed_df, chunk])
    return imputed_df

df_changed_credit_limit = impute_chunk_mean_round2(df_changed_credit_limit, chunk_size=8)
df["Changed_Credit_Limit"] = df_changed_credit_limit["Changed_Credit_Limit"]
df

/var/folders/k_/s1y99_w10p160dfzw5445lwh0000gn/T/ipykernel_80917/1630658477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_changed_credit_limit['Changed_Credit_Limit'] = df_changed_credit_limit['Changed_Credit_Limit'].replace("", np.nan)
/var/folders/k_/s1y99_w10p160dfzw5445lwh0000gn/T/ipykernel_80917/1630658477.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_changed_credit_limit['Changed_Credit_Limit'] = df_changed_credit_limit['Changed_Credit_Limit'].astype(float)


,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7.0,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,-1.0,6.0,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7.0,10.27,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,5.0,4.0,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,6.0,6.0,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,23.0,7.0,11.50,NaN,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,18.0,7.0,11.50,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,27.0,6.0,11.50,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,20.0,6.0,11.50,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Num Credit Inquiries

In [577]:
df_Credi_Inq = df[['Num_Credit_Inquiries']]
df_Credi_Inq.loc[(df_Credi_Inq['Num_Credit_Inquiries'] > 17  ),'Num_Credit_Inquiries'] = np.nan

def impute_chunk_mean_round0(df, chunk_size=8):
    imputed_df = pd.DataFrame()
    for start in range(0, len(df), chunk_size):
        chunk = df.iloc[start:start+chunk_size].copy()
        # Select numeric columns in the chunk
        num_cols = chunk.select_dtypes(include=[np.number]).columns
        if len(num_cols) > 0:
            imputer = SimpleImputer(strategy='mean')
            # Impute missing values with the mean of the non-NaN values in the chunk
            chunk[num_cols] = imputer.fit_transform(chunk[num_cols])
            # Round imputed values to the nearest unit (0 decimals)
            chunk[num_cols] = chunk[num_cols].round(0)
        imputed_df = pd.concat([imputed_df, chunk])
    return imputed_df

df_Credi_Inq = impute_chunk_mean_round0(df_Credi_Inq, chunk_size=8)

df["Num_Credit_Inquiries"] = df_Credi_Inq["Num_Credit_Inquiries"]
df

/var/folders/k_/s1y99_w10p160dfzw5445lwh0000gn/T/ipykernel_80917/3028279399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Credi_Inq.loc[(df_Credi_Inq['Num_Credit_Inquiries'] > 17  ),'Num_Credit_Inquiries'] = np.nan


,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7.0,11.27,4.0,Good,809.98,26.822620,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,-1.0,6.0,11.27,4.0,Good,809.98,31.944960,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7.0,10.27,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,5.0,4.0,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,6.0,6.0,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,23.0,7.0,11.50,3.0,Good,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,18.0,7.0,11.50,3.0,Good,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,27.0,6.0,11.50,3.0,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,20.0,6.0,11.50,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,251.67258219721603


### Credit Utilization Ratio


In [578]:
df_credit_utilization_ratio = pd.DataFrame(df['Credit_Utilization_Ratio'].round(2))
df_credit_utilization_ratio = impute_chunk_mean_round2(df_credit_utilization_ratio, chunk_size=8)
df["Credit_Utilization_Ratio"] = df_credit_utilization_ratio["Credit_Utilization_Ratio"]
df

,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7.0,11.27,4.0,Good,809.98,26.82,22 Years and 1 Months,No,80.41529543900253
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,-1.0,6.0,11.27,4.0,Good,809.98,31.94,22 Years and 2 Months,No,118.28022162236736
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7.0,10.27,4.0,Good,809.98,28.61,22 Years and 3 Months,No,81.699521264648
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,5.0,4.0,6.27,4.0,Good,809.98,26.56,22 Years and 4 Months,No,199.4580743910713
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,6.0,6.0,11.27,4.0,Good,809.98,24.80,22 Years and 5 Months,No,41.420153086217326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,23.0,7.0,11.50,3.0,Good,502.38,34.66,31 Years and 6 Months,No,60.97133255718485
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,18.0,7.0,11.50,3.0,Good,502.38,40.57,31 Years and 7 Months,No,54.18595028760385
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,27.0,6.0,11.50,3.0,Good,502.38,41.26,31 Years and 8 Months,No,24.02847744864441
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,20.0,6.0,11.50,3.0,Good,502.38,33.64,31 Years and 9 Months,No,251.67258219721603


### Amount invested monthly

In [ ]:
df_amount_invested_monthly = pd.DataFrame(pd.to_numeric(df['Amount_invested_monthly'], errors='coerce').round(2))
df_amount_invested_monthly = impute_chunk_mean_round2(df_amount_invested_monthly, chunk_size=8)
df["Amount_invested_monthly"] = df_amount_invested_monthly["Amount_invested_monthly"]
df


,Customer_ID,Month,Name,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly
0,CUS0xd40,1,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7.0,11.27,4.0,Good,809.98,26.82,22 Years and 1 Months,No,80.42
1,CUS0xd40,2,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,-1.0,6.0,11.27,4.0,Good,809.98,31.94,22 Years and 2 Months,No,118.28
2,CUS0xd40,3,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,3.0,7.0,10.27,4.0,Good,809.98,28.61,22 Years and 3 Months,No,81.70
3,CUS0xd40,4,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,5.0,4.0,6.27,4.0,Good,809.98,26.56,22 Years and 4 Months,No,199.46
4,CUS0xd40,5,Aaron Maashoh,Lonton,Oxford Street,23.0,Manager,19114.12,1824.84,3.0,...,6.0,6.0,11.27,4.0,Good,809.98,24.80,22 Years and 5 Months,No,41.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS0x942c,4,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,23.0,7.0,11.50,3.0,Good,502.38,34.66,31 Years and 6 Months,No,60.97
99996,CUS0x942c,5,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,18.0,7.0,11.50,3.0,Good,502.38,40.57,31 Years and 7 Months,No,54.19
99997,CUS0x942c,6,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,27.0,6.0,11.50,3.0,Good,502.38,41.26,31 Years and 8 Months,No,24.03
99998,CUS0x942c,7,Nicks,BadShire,North Street,25.0,Cleaner,39628.99,3359.42,4.0,...,20.0,6.0,11.50,3.0,Good,502.38,33.64,31 Years and 9 Months,No,251.67


In [ ]:
nan_counts = df.isna().sum()
print("Missing values by column:")
print(nan_counts)

Missing values by column:
Customer_ID                 0
Month                       0
Name                        0
City                        0
Street                      0
Age                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Type_of_Loan                0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Amount_invested_monthly     0
dtype: int64


--- 
## EDA (Post Data Handling)

--- 
## Class Imbalance Assesment

--- 
## Training the Credit Score Prediction model

--- 
## Results Analysis

--- 
## Dashboard